In [2]:
import re
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 1. 설정 및 맵 정의
BRAND_ORIGIN_MAP = {
    "기아": "국산", "현대": "국산", "제네시스": "국산", "KGM": "국산", "르노코리아": "국산",
    "벤츠": "해외", "BMW": "해외", "비엠더블유": "해외", "테슬라": "해외", "볼보": "해외", "BYD": "해외",
}

URL = "https://www.car.go.kr/ri/stat/list.do"
LAST_PAGE = 10  # 수집하고 싶은 페이지 수
TITLE_CSS = "ul.board-hrznt-list li a strong"
PAGE_INFO_CSS = "p.count-result"

brand_re = re.compile(r"\[(.*?)\]")

def clean(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

def wait_page_loaded(driver, wait, page_no: int):
    pattern = rf"페이지\s*{page_no}\s*/\s*\d+"
    wait.until(lambda d: re.search(pattern, d.find_element(By.CSS_SELECTOR, PAGE_INFO_CSS).text))

def go_page(driver, wait, page_no: int):
    driver.execute_script(f"$main.event.fn_search({page_no});")
    wait_page_loaded(driver, wait, page_no)
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, TITLE_CSS)) > 0)

def get_brand_title_list(driver):
    titles = driver.find_elements(By.CSS_SELECTOR, TITLE_CSS)
    out = []
    for el in titles:
        title = clean(el.text)
        m = brand_re.search(title)
        brand = m.group(1).strip() if m else None # 공백 제거 추가
        out.append((brand, title))
    return out

def main():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    wait = WebDriverWait(driver, 15)

    results = []  # 데이터 저장 리스트

    try:
        driver.get(URL)
        wait_page_loaded(driver, wait, 1)

        for page in range(1, LAST_PAGE + 1):
            if page != 1:
                go_page(driver, wait, page)

            rows = get_brand_title_list(driver)

            for idx, (brand, title) in enumerate(rows, start=1):
                # 맵에 존재하는 브랜드만 필터링
                if brand in BRAND_ORIGIN_MAP:
                    origin = BRAND_ORIGIN_MAP[brand]
                    results.append((page, idx, origin, brand, title))

            print(f"[{page}/{LAST_PAGE}] 현재까지 조건 매칭 데이터: {len(results)}개 수집됨")
            time.sleep(0.3)

    except Exception as e:
        print(f"에러 발생: {e}")

    finally:
        driver.quit()

    # --- 여기서부터 출력 및 저장 (main 함수 내부) ---
    if not results:
        print("수집된 데이터가 없습니다. BRAND_ORIGIN_MAP이나 페이지를 확인하세요.")
        return

    # 1. 콘솔에 전체 결과 출력
    print("\n" + "="*80)
    print(f"{'페이지':<5} | {'순번':<5} | {'구분':<5} | {'브랜드':<12} | {'제목'}")
    print("-" * 80)

    for r in results:
        print(f"{r[0]:<7} | {r[1]:<7} | {r[2]:<7} | {r[3]:<14} | {r[4]}")

    print("-" * 80)
    print(f"총 수집 건수: {len(results)}건")

    # 2. CSV 파일로 저장
    file_name = "car_recall_results.csv"
    try:
        with open(file_name, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["페이지", "순번", "국산/해외", "브랜드", "제목"])
            writer.writerows(results)
        print(f"✅ 성공! 전체 데이터가 '{file_name}'에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 에러 발생: {e}")

if __name__ == "__main__":
    main()

[1/10] 현재까지 조건 매칭 데이터: 3개 수집됨
[2/10] 현재까지 조건 매칭 데이터: 3개 수집됨
[3/10] 현재까지 조건 매칭 데이터: 4개 수집됨
[4/10] 현재까지 조건 매칭 데이터: 7개 수집됨
[5/10] 현재까지 조건 매칭 데이터: 12개 수집됨
[6/10] 현재까지 조건 매칭 데이터: 12개 수집됨
[7/10] 현재까지 조건 매칭 데이터: 13개 수집됨
[8/10] 현재까지 조건 매칭 데이터: 15개 수집됨
[9/10] 현재까지 조건 매칭 데이터: 18개 수집됨
[10/10] 현재까지 조건 매칭 데이터: 20개 수집됨

페이지   | 순번    | 구분    | 브랜드          | 제목
--------------------------------------------------------------------------------
1       | 1       | 해외      | 비엠더블유          | [비엠더블유] BMW i5 eDrive40 등 13차종 - BCP 컨트롤 유닛(전동식 에어컨 컴프레셔 제어장치) 관련 리콜
1       | 2       | 해외      | 비엠더블유          | [비엠더블유] BMW 220i Active Tourer 등 7차종 - 통합제동장치 관련 리콜
1       | 3       | 해외      | 벤츠             | [벤츠] Mercedes-AMG E 53 Hybrid 4MATIC+ - 변속기 배선 관련 리콜
3       | 2       | 해외      | 볼보             | [볼보] S90 등 7차종 - 긴급제동장치(AEB) 관련 리콜
4       | 1       | 해외      | 벤츠             | [벤츠] AMG G 63 - 엔진 컨트롤 유닛 관련 리콜
4       | 4       | 해외      | 볼보             | [볼보] FH 카고 - 앞차축 고정볼트 관련 리콜
4       | 5       